<a href="https://colab.research.google.com/github/guilhermeacosta/ProjetoETL/blob/main/Projeto_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fonte de dados:** https://www.kaggle.com/datasets/datacertlaboratoria/projeto-3-segmentao-de-clientes-no-ecommerce

O projeto de ETL escolhido foi baseado no desafio do Kaggle do link a cima, mas simplifiquei bastante, em resumo a idéia é:

Trabalhamos com um escritório de Mentoria de pequenos negócios, e recebemos uma demanda de um cliente novo, este cliente não organiza muito bem seus dados de vendas, por este motivo, nosso superior pede que a primeira coisa a ser feita é uma limpeza nos dados de vendas deste cliente e algumas segmentações e filtragens.

O objetivo será entender melhor quem é o cliente deste estabelecimento

- Este cliente volta a comprar?
- Quem é o melhor cliente?
- De onde são os clientes?

Os processos serão divididos na extração de um arquivo csv

A limpeza dos dados (removendo alguns dados repetidos, excluindo dados que não fazem sentido, e etc)

realizar algumas filtragens para preparar uma analize de clientes.

por fim gerar um arquivo de excel com algumas abas com essas segmentações de informação

### ✅ Importar o pandas e realizar a leitura dos dados recebidos

In [102]:
import pandas as pd

data = pd.read_csv('./vendas-por-fatura.csv')

In [103]:
data.columns = ['Num_Fatura','Data_Fatura','ID_Cliente','País','Qnt','Valor']
data.head()

,Num_Fatura,Data_Fatura,ID_Cliente,País,Qnt,Valor
0,548370,3/30/2021 16:14:00,15528.0,United Kingdom,123,"229,33"
1,575767,11/11/2021 11:11:00,17348.0,United Kingdom,163,"209,73"
2,C570727,10/12/2021 11:32:00,12471.0,Germany,-1,"-1,45"
3,549106,4/6/2021 12:08:00,17045.0,United Kingdom,1,"39,95"
4,573112,10/27/2021 15:33:00,16416.0,United Kingdom,357,"344,83"


### ✅ Realizar algumas checagens iniciais para identificar possiveis correções

In [104]:
data.shape

(25953, 6)

In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25953 entries, 0 to 25952
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Num_Fatura   25953 non-null  object 
 1   Data_Fatura  25953 non-null  object 
 2   ID_Cliente   22229 non-null  float64
 3   País         25953 non-null  object 
 4   Qnt          25953 non-null  int64  
 5   Valor        25953 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.2+ MB


In [106]:
data.nunique() #Identifica valores unicos

Num_Fatura     25900
Data_Fatura    23260
ID_Cliente      4372
País              38
Qnt             1853
Valor          17540
dtype: int64

In [107]:
data.isnull().sum() #Identifica registros sem alguma informação

Num_Fatura        0
Data_Fatura       0
ID_Cliente     3724
País              0
Qnt               0
Valor             0
dtype: int64

In [108]:
data.duplicated().sum() #Identifica registros duplicados

10

### ✅ Realizar o tratamento dos erros encontrados:

In [109]:
# Realizar o tratamento dos erros encontrados

In [110]:
clientes_nao_identificados = data[data.ID_Cliente.isnull()] # Por pura curiosidade, queria ver quantos registros sem cliente informado tinhamos,
clientes_nao_identificados.shape                            # considerando o Num_Fatura são arquivos unicos, como o propósito é conhecer o cliente,
                                                            # vamos excluir estes valores.

(3724, 6)

In [111]:
data = data.dropna(subset=['ID_Cliente']) # Removendo aquivos identificados acima.
data.shape

(22229, 6)

In [112]:
data.isnull().sum() # checando se deu tudo certo.

Num_Fatura     0
Data_Fatura    0
ID_Cliente     0
País           0
Qnt            0
Valor          0
dtype: int64

In [113]:
data.drop_duplicates(inplace=True) # Simples, removemos aqui os arquivos duplicados que identificamos em nossa exploração dos dados
data.shape                         # Ja aproveitamos para conferir se os registros foram expluidos

                                   # OBS: Na checagem identificamos 10 dados mas apenas 8 arquivos foram excluidos, o motivo disto é
                                   # que tinhamos arquivos duplicados entre os registro sem o ID do cliente, eliminados anteriormente.

(22221, 6)

### Removendo Devoluções

No arquivo temos a identificação do numero da nota, alguns iniciam com a letra "C", estes são devoluções, Para simplificar nossa limpeza pensei anteriormente em exclui-los do nosso projeto, mas antes fiz uma pequena checagem para ver se vale a pena destacar algum comportamento padrão de alguns clientes em especificos e acabei me interessando em registrar a informação encontrada para analize futura.

In [114]:
devolucao = data.loc[data['Num_Fatura'].str.startswith('C')]
devolucao.shape

(3655, 6)

In [115]:
clientes_que_devolvem = devolucao['ID_Cliente'].value_counts() #Contagem de devoluções por cliente
clientes_que_devolvem.head(10)

14911.0    47
17841.0    45
14606.0    35
14527.0    31
15311.0    27
16422.0    24
13089.0    21
12471.0    19
13408.0    19
13534.0    18
Name: ID_Cliente, dtype: int64

In [116]:
data['ID_Cliente'].value_counts() # Contagem de compras e devoluções por cliente,
                                  # comparando as informações com a da cedula anterior, percebemos que em um mesmo periodo,
                                  # alguns clientes tem indices altos de devolução de peças, como o objetivo não é realizar uma analise e sim um projeto ETL,
                                  # vou apenas registrar isso em um arquivo para uma futura checagem.

14911.0    249
12748.0    225
17841.0    169
14606.0    130
15311.0    118
          ... 
17839.0      1
12401.0      1
17222.0      1
15185.0      1
13600.0      1
Name: ID_Cliente, Length: 4372, dtype: int64

In [117]:
devolucao_busca = data['Num_Fatura'].str.startswith('C')
data.drop(data[devolucao_busca].index, inplace=True)

In [118]:
data.shape

(18566, 6)

### ✅ Por Fim alterar o tipo de dado da coluna 'Data_Fatura' e 'Valor'

In [119]:
data['Data_Fatura'] = pd.to_datetime(data['Data_Fatura'])
data['Valor'] = data['Valor'].str.replace(',', '.')
data['Valor'] = pd.to_numeric(data['Valor'])

In [120]:
data.sort_values(by=['Data_Fatura'], inplace= True)
data.head()

,Num_Fatura,Data_Fatura,ID_Cliente,País,Qnt,Valor
9367,536365,2020-12-01 08:26:00,17850.0,United Kingdom,40,139.12
18259,536366,2020-12-01 08:28:00,17850.0,United Kingdom,12,22.20
11185,536368,2020-12-01 08:34:00,13047.0,United Kingdom,15,70.05
6876,536367,2020-12-01 08:34:00,13047.0,United Kingdom,83,278.73
8195,536369,2020-12-01 08:35:00,13047.0,United Kingdom,3,17.85


###  Preparando para salvar nossas informações em um arquivo excel

In [121]:
#Clientes_que_devolvem
#data


Clientes_por_ordem_de_numero_de_compra = data['ID_Cliente'].value_counts()
paises_com_mais_compras = data['País'].value_counts()



In [122]:
clientes_df = data.filter(items=['ID_Cliente', 'Valor'])
clientes_valor_medio = clientes_df.groupby('ID_Cliente').mean()

In [123]:
paises_df = data.filter(items=['País', 'ID_Cliente'])
paises_df = paises_df.drop_duplicates('ID_Cliente')
paises_por_numero_de_clientes = paises_df.groupby('País').count().sort_values(by='ID_Cliente', ascending= False)
paises_por_numero_de_clientes.head(40)

,ID_Cliente
País,
United Kingdom,3921
Germany,94
France,87
Spain,28
Belgium,24
Switzerland,20
Portugal,19
Italy,14
Finland,12


### 💼 Registrar os dados em uma planilha de excel

In [124]:
with pd.ExcelWriter('Dados_limpos.xlsx') as writer:

    data.to_excel(writer, sheet_name= 'Vendas_Geral', header=True, index= False)

    Clientes_por_ordem_de_numero_de_compra.to_excel(writer, sheet_name= 'Clientes_por_num_compras', header=True, index= True)

    clientes_valor_medio.to_excel(writer, sheet_name= 'Clientes_por_val_med_gasto', header=True, index= True)

    paises_com_mais_compras.to_excel(writer, sheet_name= 'paises_por_n_venda', header=True, index= True)

    paises_por_numero_de_clientes.to_excel(writer, sheet_name= 'paises_por_n_clientes', header=True, index= True)

    clientes_que_devolvem.to_excel(writer, sheet_name= 'Clientes_por_n_devolucoes', header=True, index= True)